# [完整代码](#full-code)

# Import Libraries

In [1]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime
import pandas as pd

In [2]:
# engine= create_engine('sqlite:///../../dataset/us/us_ticker_price_yf.db')
engine= create_engine('sqlite:///../../dataset/us/us_ticker_seven_year_price.db')

# Gather Data

In [3]:
df = pd.read_sql('INTC', engine)
# df.rename(columns={'hgih':'high'},inplace=True)
# df['chg_pct'] = ((df.close - df.open)/df.open)*100
# df['close'] = df['close'].apply(lambda x: round(x,3))
df

,Open,High,Low,Close,Volume,datetime
0,23.755705,24.267113,23.664972,23.986664,38665900.0,2015-09-03
1,23.565984,23.838185,23.434008,23.524742,29116500.0,2015-09-04
2,23.904177,24.333099,23.879432,24.333099,27805000.0,2015-09-08
3,24.514565,24.844507,24.060898,24.118637,30774100.0,2015-09-09
4,24.069146,24.465074,23.838188,24.143383,25195800.0,2015-09-10
...,...,...,...,...,...,...
1758,33.230000,33.410000,32.810001,32.939999,32976800.0,2022-08-29
1759,33.040001,33.070000,32.150002,32.259998,40182500.0,2022-08-30
1760,32.389999,32.389999,31.770000,31.920000,44074500.0,2022-08-31
1761,31.570000,31.830000,31.129999,31.760000,36440700.0,2022-09-01


In [4]:
# adf = pd.read_sql('{}'.format(ticker_name), engine)
ticker_name = 'INTC'
target_date = '2019-10-18'
time_zone = 140
extrem_zone = 10
df['Close'] = df['Close'].apply(lambda x: round(x,3))
df['VMA20'] = ta.SMA(df.Volume, timeperiod=20)
end_index = df[df.datetime == '{}'.format(target_date)].index.values[0] + 1
start_index = end_index - time_zone
df = df[start_index:end_index]
df.index = range(len(df))

# 关闭警告的设置方法
pd.set_option('mode.chained_assignment', None)
# 判断是不是前后5天的极值点
df['extremum'] = 0
for i in range(extrem_zone,len(df)):
    if df.Close.iloc[i] == df.Close.iloc[i-extrem_zone:i+extrem_zone].min() or df.Close.iloc[i] == df.Close.iloc[i-5:i+5].max() :
        df['extremum'].iloc[i] = 1
df['extremum_val'] = 0
for i in range(len(df)):
    if df.iloc[i].extremum == 1:
        df.extremum_val.iloc[i] = round(df.iloc[i].Low,3)
    elif df.iloc[i].extremum == 2:
        df.extremum_val.iloc[i] = round(df.iloc[i].High,3)

In [5]:
df[df.extremum == 1] 

,Open,High,Low,Close,Volume,datetime,VMA20,extremum,extremum_val
13,52.933434,53.552432,52.687655,53.543,12327500.0,2019-04-22,17703485.0,1,52.688
42,40.527078,40.838473,39.565419,39.804,31155700.0,2019-06-03,30011880.0,1,39.565
48,43.283830,43.338783,42.606091,42.908,18415500.0,2019-06-11,24888735.0,1,42.606
59,43.641021,44.401188,43.558593,44.136,21886300.0,2019-06-26,20674415.0,1,43.559
64,44.126430,44.474460,43.961573,44.438,10739100.0,2019-07-03,19476870.0,1,43.962
78,47.606716,48.513425,47.441860,48.468,20896400.0,2019-07-24,18795395.0,1,47.442
96,43.760480,43.889566,43.373220,43.548,21403600.0,2019-08-19,25857770.0,1,43.373
100,42.737009,42.995185,41.307833,41.455,32814000.0,2019-08-23,25265195.0,1,41.308
113,48.868631,49.172909,48.011124,48.878,23308700.0,2019-09-12,20667190.0,1,48.011
125,47.199728,47.577769,46.969216,47.513,13652100.0,2019-09-30,19397790.0,1,46.969


In [6]:
index_arr = np.array(df.query('extremum != 0').index)
price_arr = np.array(df[df.extremum_val != 0].extremum_val)

In [11]:
df.tail(1).Close.values[0]

47.356

In [7]:
while True:    
    min1_val = np.min(price_arr)
    min1_pos = np.argmin(price_arr)
    min1_ind = index_arr[min1_pos]
    max1_val = round(df.iloc[:min1_ind].max().values[0],3)

    if price_arr[min1_pos+1:].size == 0:
        # print('{} is not vcp in {}'.format(ticker_name,target_date))
        break
    
    min2_val = np.min(price_arr[min1_pos+1:])
    min2_pos = np.argmin(price_arr[min1_pos+1:]) + min1_pos + 1
    min2_ind = index_arr[min2_pos]
    max2_val = round(df.iloc[min1_ind+1:min2_ind].max().values[0],3)

    if price_arr[min2_pos+1:].size == 0:
        # print('{} is not vcp in {}'.format(ticker_name, target_date))
        break

    min3_val = np.min(price_arr[min2_pos+1:])
    min3_pos = np.argmin(price_arr[min2_pos+1:]) + min2_pos + 1
    min3_ind = index_arr[min3_pos]
    max3_val = round(df.iloc[min2_ind+1:min3_ind].max().values[0],3)

    dec1 = (max1_val - min1_val) / max1_val
    dec2 = (max2_val - min2_val) / max2_val
    dec3 = (max3_val - min3_val) / max3_val

    vol1 = df.iloc[min1_ind].VMA20
    vol2 = df.iloc[min2_ind].VMA20
    vol3 = df.iloc[min3_ind].VMA20

    if max(dec1, dec2) > dec3 and max(vol1, vol2) > vol3 and min3_ind > int(time_zone * 0.67) and [max(max2_val, max3_val, df.tail(1).Close) - min(max2_val, max3_val, df.tail(1).Close)] / max(max2_val, max3_val, df.tail(1).Close) < 0.15:
        print('{} is vcp in {}'.format(ticker_name,target_date))
        # print('dec1:{} dec2:{} dec3:{}'.format(dec1,dec2,dec3))
        # print('vol1:{} vol2:{} vol3:{}'.format(vol1,vol2,vol3))
        # date1 = df.iloc[min1_ind].datetime
        # date2 = df.iloc[min2_ind].datetime
        # date3 = df.iloc[min3_ind].datetime
        # print('min1:{} min2:{} min3:{}'.format(date1,date2,date3) )
        # print('min3_ind:{}'.format(min3_ind))
        break
        # return True
    else:
        print('{} is not vcp in {}'.format(ticker_name, target_date))
        print('dec1:{} dec2:{} dec3:{}'.format(dec1,dec2,dec3))
        print('vol1:{} vol2:{} vol3:{}'.format(vol1,vol2,vol3))
        date1 = df.iloc[min1_ind].datetime
        date2 = df.iloc[min2_ind].datetime
        date3 = df.iloc[min3_ind].datetime
        print('min1:{} min2:{} min3:{}'.format(date1,date2,date3) )
        print('min3_ind:{}'.format(min3_ind))
        break

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [13]:
df[df[df.datetime =='2020-09-15'].index.values[0]:]

,Open,High,Low,Close,Volume,datetime
1266,46.967714,47.731798,46.911114,47.165810,27598300.0,2020-09-15
1267,47.646903,48.137428,47.326175,47.514839,26531400.0,2020-09-16
1268,46.571519,47.628036,46.505487,47.467670,26959900.0,2020-09-17
1269,47.477111,47.637473,46.260232,47.062050,50123500.0,2020-09-18
1270,46.590395,46.929990,46.062140,46.901691,27377100.0,2020-09-21
...,...,...,...,...,...,...
1758,33.230000,33.410000,32.810001,32.939999,32976800.0,2022-08-29
1759,33.040001,33.070000,32.150002,32.259998,40182500.0,2022-08-30
1760,32.389999,32.389999,31.770000,31.920000,44074500.0,2022-08-31
1761,31.570000,31.830000,31.129999,31.760000,36440700.0,2022-09-01


# Main Function

## Set Time

In [10]:
# 140约等于200天
df = df[9834:9984]
df.index = range(len(df))
df_length = len(df)

## Gather Data

In [11]:
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
# 判断是不是10天内的极值点
df['extremum'] = 0
for i in range(5,df_length):
    # or df.close.loc[i] == df.close.loc[i-5:i+5].max() 
    if df.close.loc[i] == df.close.loc[i-5:i+5].min():
        df['extremum'].loc[i] = 1
        
minimum_index = np.array(df[df['extremum']==1].index)
minimum_value = np.array(df[df['extremum']==1].close)

C:\Users\Rain\AppData\Local\Temp\ipykernel_26416\2692695270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
C:\Users\Rain\AppData\Local\Temp\ipykernel_26416\2692695270.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['extremum'] = 0
C:\Users\Rain\AppData\Local\Temp\ipykernel_26416\2692695270.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [12]:
df

,open,high,low,close,Adj Close,volume,datetime,VMA20,extremum
0,54.419998,54.509998,53.869999,54.10,49.743919,16274000,2019-03-18,NaN,0
1,54.470001,54.650002,54.000000,54.17,49.808285,18274800,2019-03-19,NaN,0
2,54.060001,54.400002,53.610001,53.82,49.486473,24325300,2019-03-20,NaN,0
3,53.919998,54.990002,53.840000,54.64,50.240452,19127200,2019-03-21,NaN,0
4,54.400002,54.619999,53.209999,53.26,48.971558,21438400,2019-03-22,NaN,0
...,...,...,...,...,...,...,...,...,...
145,52.000000,52.790001,51.820000,52.09,48.514450,24072500,2019-10-11,17480585.0,0
146,51.900002,51.950001,51.439999,51.64,48.095337,15016800,2019-10-14,17563695.0,0
147,51.930000,53.070000,51.830002,52.65,49.036015,18878100,2019-10-15,17525545.0,0
148,52.349998,52.560001,52.119999,52.45,48.849743,14769300,2019-10-16,17518670.0,0


In [60]:
# find the min1
min1_val = np.min(minimum_value)
min1_pos = np.argmin(minimum_value)
min1_ind = minimum_index[min1_pos]
# find the max1
max1_val = df.iloc[:min1_ind].close.max()

# min2
min2_val = np.min(minimum_value[min1_pos+1:])
min2_pos = np.argmin(minimum_value[min1_pos+1:]) + min1_pos + 1
min2_ind = minimum_index[min2_pos]
# max2
max2_val = df.iloc[min1_ind:min2_ind].close.max()

# min3
min3_val = np.min(minimum_value[min2_pos+1:])
min3_pos = np.argmin(minimum_value[min2_pos+1:]) + min2_pos + 1
min3_ind = minimum_index[min3_pos]
# max3
max3_val = df.iloc[min2_ind:min3_ind].close.max()

## Select Condition

### prepare data

In [65]:
vol1 = df.iloc[min1_ind].VMA20
vol2 = df.iloc[min2_ind].VMA20
vol3 = df.iloc[min3_ind].VMA20

In [66]:
vol1, vol2, vol3

(30011880.0, 25265195.0, 19399655.0)

In [67]:
dec1 = (max1_val - min1_val) / max1_val
dec2 = (max2_val - min2_val) / max2_val
dec3 = (max3_val - min3_val) / max3_val

In [68]:
dec1,dec2,dec3

(0.2611356681400884, 0.15041572184429328, 0.06828900207508012)

In [70]:
# 
if vol1 > vol2 > vol3 and dec1 > dec2 >dec3 and max2_val*1.05 > max3_val:
    print('ticker is vcp')
else:
    print('not vcp pattern')

ticker is vcp


<h1><span id="full-code">Full code</span></h1>

In [ ]:
# 获取Volume MA
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
# 判断是不是10天内的极值点
df['extremum'] = 0
for i in range(5,df_length):
    # or df.close.loc[i] == df.close.loc[i-5:i+5].max() 
    if df.close.loc[i] == df.close.loc[i-5:i+5].min():
        df['extremum'].loc[i] = 1
        
minimum_index = np.array(df[df['extremum']==1].index)
minimum_value = np.array(df[df['extremum']==1].close)

# Main
# 140约等于200天
df = df[9834:9984]
df.index = range(len(df))
df_length = len(df)

# find the min1
min1_val = np.min(minimum_value)
min1_pos = np.argmin(minimum_value)
min1_ind = minimum_index[min1_pos]
# find the max1
max1_val = df.iloc[:min1_ind].close.max()

# min2
min2_val = np.min(minimum_value[min1_pos+1:])
min2_pos = np.argmin(minimum_value[min1_pos+1:]) + min1_pos + 1
min2_ind = minimum_index[min2_pos]
# max2
max2_val = df.iloc[min1_ind:min2_ind].close.max()

# min3
min3_val = np.min(minimum_value[min2_pos+1:])
min3_pos = np.argmin(minimum_value[min2_pos+1:]) + min2_pos + 1
min3_ind = minimum_index[min3_pos]
# max3
max3_val = df.iloc[min2_ind:min3_ind].close.max()

vol1 = df.iloc[min1_ind].VMA20
vol2 = df.iloc[min2_ind].VMA20
vol3 = df.iloc[min3_ind].VMA20

dec1 = (max1_val - min1_val) / max1_val
dec2 = (max2_val - min2_val) / max2_val
dec3 = (max3_val - min3_val) / max3_val

# 
if vol1 > vol2 > vol3 and dec1 > dec2 >dec3 and max2_val*1.05 > max3_val:
    print('ticker is vcp')
else:
    print('not vcp pattern')